In [1]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())



# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

app_train_def = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v1.csv')
app_train_def_2 = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v2.csv')
print(app_train_def.info())

1.23.5
Directorio actual: c:\Users\Yeray\Desktop\DATA_SCIENCE_ML\Home-Credit-TFG\JUPYTER_NOTEBOOKS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 307 entries, TARGET to WALLSMATERIAL_MODE_Wooden
dtypes: bool(127), float64(138), int64(42)
memory usage: 459.5 MB
None


In [2]:
#Train y Test
X = app_train_def_2.drop(columns = ['TARGET'])
y = app_train_def_2['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [3]:
#Definimos espacio de hiperparámetros para LightGBM
space_lightGBM = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo

    'num_leaves': hp.quniform('num_leaves', 20, 40, 1), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 50, 90, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.02),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.001, 0.1),
    'max_bin': hp.quniform('max_bin', 250, 350, 10),
    'feature_fraction': hp.uniform('feature_fraction', 0.2, 0.5),
    'max_depth': hp.choice('max_depth', [-1, 3, 5]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 5, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 110),  
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 0.2),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}

Trials_lightGBM_Ligero = Trials()


In [4]:
n_iter = 200

def F_OPT(params):
    params['max_bin'] = int(params['max_bin'])
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])

    model = LGBMClassifier(
        n_estimators=n_iter,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=params['num_leaves'],
        learning_rate=params['learning_rate'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=params['max_depth'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42)
    
    cv_Strat = StratifiedKFold(n_splits=5)
    auc_lightGBM_Ligero = cross_val_score(model, X_train, y_train, cv=cv_Strat, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Ligero, 'status': STATUS_OK}
    

In [5]:
best_lightGBM_Ligero = fmin(fn=F_OPT,
                            space=space_lightGBM,
                            algo=tpe.suggest,
                            max_evals=15,
                            trials=Trials_lightGBM_Ligero,
                            rstate= np.random.default_rng(42)
)

best_lightGBM_Ligero['num_leaves'] = int(best_lightGBM_Ligero['num_leaves'])
best_lightGBM_Ligero['max_bin'] = int(best_lightGBM_Ligero['max_bin'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])
best_lightGBM_Ligero['bagging_freq'] = int(best_lightGBM_Ligero['bagging_freq'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])

print(best_lightGBM_Ligero)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

  File "C:\Users\Yeray\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



[LightGBM] [Warning] feature_fraction is set=0.4969651399430127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4969651399430127
[LightGBM] [Warning] min_gain_to_split is set=0.0655224975751405, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0655224975751405
[LightGBM] [Warning] bagging_fraction is set=0.5304174185556783, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5304174185556783
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.4969651399430127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4969651399430127
[LightGBM] [Warning] min_gain_to_split is set=0.0655224975751405, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0655224975751405
[LightGBM] [Warning] bagging_fraction is set=0.5304174185556783, subsample=1.0 will be ignored. Current value: bagging_fraction=0

In [6]:
import pickle
with open("Trials_LightGBM_Ligero_v2.pkl", "wb") as f:
    pickle.dump(Trials_lightGBM_Ligero, f)
with open("Trials_lightGBM_Ligero_v2.pkl", "rb") as f:
    Trials_lightGBM_Ligero = pickle.load(f)

In [18]:
#Mostramos los resultados de los trials 
from tabulate import tabulate

def trials_to_dataframe(trials):
    """
    Convierte la info de 'trials' de Hyperopt en un DataFrame 
    con columnas relevantes (loss, hiperparámetros, etc.).
    """
    rows = []
    for trial in trials.trials:
        # trial['result']['loss'] -> la métrica
        # trial['misc']['vals'] -> diccionario de hiperparámetros propuestos
        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        # Convertir vals a algo "plano"
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    # df con columnas = keys (vals + 'loss')
    df = pd.DataFrame(rows)
    
    # A veces los hps que eran hp.quniform se quedan en listas => df['num_leaves'].apply(lambda x: x[0] if isinstance(x, list) else x)
    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)
    
    return df

# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Ligero)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

+----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
|  7 |           0.506587 |              1 |           0.416329 |       0.0282566 |       280 |           2 |                  80 |         0.00365639 |          0.034361   |           35 |  0.166018   |      7.07155 | -0.765068 |
| 12 |           0.698999 |              3 |           0.368491 |       0.02

In [9]:
#Vamos a realizar una selección de variables previa a una optimización más completa y una búsqueda SS
# más exhaustiva de hiperparámetros. Para ello, vamos a utilizar Permutation Importance.
from sklearn.inspection import permutation_importance

params_ligero = best_lightGBM_Ligero.copy()

model_ligero = LGBMClassifier(
    n_estimators=n_iter,          # = 200 (tu fase “ligera”)
    boosting_type='gbdt',         # Fijo
    objective='binary',           # Fijo
    metric='auc',                 # Fijo
    # Luego pasas los HP que has encontrado en la búsqueda
    num_leaves=params_ligero['num_leaves'],
    learning_rate=params_ligero['learning_rate'],
    min_child_samples=params_ligero['min_child_samples'],
    min_child_weight=params_ligero['min_child_weight'],
    min_gain_to_split=params_ligero['min_gain_to_split'],
    max_bin=params_ligero['max_bin'],
    feature_fraction=params_ligero['feature_fraction'],
    max_depth=params_ligero['max_depth'],
    bagging_fraction=params_ligero['bagging_fraction'],
    bagging_freq=params_ligero['bagging_freq'],
    reg_lambda=params_ligero['reg_lambda'],
    reg_alpha=params_ligero['reg_alpha'],
    scale_pos_weight=1,           # Fijo
    is_unbalance=False,           # Fijo
    random_state=42
)

model_ligero.fit(X_train, y_train)


[LightGBM] [Warning] feature_fraction is set=0.41632922310880127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41632922310880127
[LightGBM] [Warning] min_gain_to_split is set=0.03436099418526572, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.03436099418526572
[LightGBM] [Warning] bagging_fraction is set=0.506587413206087, subsample=1.0 will be ignored. Current value: bagging_fraction=0.506587413206087
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.41632922310880127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41632922310880127
[LightGBM] [Warning] min_gain_to_split is set=0.03436099418526572, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.03436099418526572
[LightGBM] [Warning] bagging_fraction is set=0.506587413206087, subsample=1.0 will be ignored. Current value: bagging_fract

LGBMClassifier(bagging_fraction=0.506587413206087, bagging_freq=1,
               feature_fraction=0.41632922310880127, is_unbalance=False,
               learning_rate=0.028256630457929727, max_bin=280, max_depth=2,
               metric='auc', min_child_samples=80,
               min_child_weight=0.003656393023740737,
               min_gain_to_split=0.03436099418526572, n_estimators=200,
               num_leaves=35, objective='binary', random_state=42,
               reg_alpha=0.1660179264744961, reg_lambda=7.071554964050417,
               scale_pos_weight=1)

In [10]:
pi = permutation_importance(
    estimator=model_ligero,
    X=X_train,
    y=y_train,
    scoring='roc_auc',
    n_repeats=5,
    random_state=42
)

[LightGBM] [Warning] feature_fraction is set=0.41632922310880127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41632922310880127
[LightGBM] [Warning] min_gain_to_split is set=0.03436099418526572, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.03436099418526572
[LightGBM] [Warning] bagging_fraction is set=0.506587413206087, subsample=1.0 will be ignored. Current value: bagging_fraction=0.506587413206087
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.41632922310880127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41632922310880127
[LightGBM] [Warning] min_gain_to_split is set=0.03436099418526572, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.03436099418526572
[LightGBM] [Warning] bagging_fraction is set=0.506587413206087, subsample=1.0 will be ignored. Current value: bagging_fract

In [14]:
with open("pi_lightGBM_Ligero_v2.pkl", "rb") as f:
    pi = pickle.load(f)


In [19]:


features = X_train.columns
importances_mean = pi.importances_mean
importances_std = pi.importances_std

#Lo unimos en un DF

df_pi = pd.DataFrame({
    'feature': features,
    'importance_mean': importances_mean,
    'importance_std': importances_std
})
df_pi.to_csv("Permutation_Importance_LightGBM_Ligero.csv", index=False)

df_pi_sorted = df_pi.sort_values(by='importance_mean', ascending=False)
print(tabulate(df_pi_sorted, headers='keys', tablefmt='psql'))

+-----+-----------------------------------------------------------------------------------------+-------------------+------------------+
|     | feature                                                                                 |   importance_mean |   importance_std |
|-----+-----------------------------------------------------------------------------------------+-------------------+------------------|
| 532 | EXT_SOURCE_MEAN                                                                         |       0.024893    |      0.000922636 |
| 511 | CREDIT_GOODS_PRICE_RATIO                                                                |       0.00297741  |      0.000103653 |
| 173 | AMT_CREDIT_SUM_DEBT_TO_AMT_CREDIT_SUM_RATIO_MAX_BUREAU                                  |       0.00196032  |      2.69763e-05 |
| 509 | CREDIT_ANNUITY_RATIO                                                                    |       0.00165066  |      4.57911e-05 |
| 531 | EXT_SOURCE_SUM                   

In [43]:
subset_features_pi = df_pi_sorted[df_pi_sorted['importance_mean'] > 0.0]
subset_features= subset_features_pi['feature'].tolist()
X_train_subset = X_train[subset_features]
y_train_subset = y_train
X_test_subset = X_test[subset_features]
y_test_subset = y_test
print(X_train_subset.describe())

       EXT_SOURCE_MEAN  CREDIT_GOODS_PRICE_RATIO  \
count    246008.000000             246008.000000   
mean          0.509119                  0.166322   
std           0.149909                  0.021177   
min           0.000011                  0.000000   
25%           0.413676                  0.145299   
50%           0.524158                  0.165607   
75%           0.622792                  0.179145   
max           0.878903                  1.000000   

       AMT_CREDIT_SUM_DEBT_TO_AMT_CREDIT_SUM_RATIO_MAX_BUREAU  \
count                                      246008.000000        
mean                                            0.000033        
std                                             0.002902        
min                                             0.000000        
25%                                             0.000013        
50%                                             0.000018        
75%                                             0.000019        
max        

In [44]:
space_lightGBM_completo = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo

    #'num_leaves': hp.quniform('num_leaves', 20, 40, 1), 
    #'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 60, 85, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.01),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.01, 0.1),
    'max_bin': hp.quniform('max_bin', 250, 300, 5),
    'feature_fraction': hp.uniform('feature_fraction', 0.2, 0.45),
    #'max_depth': hp.choice('max_depth', [-1, 3]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.3, 0.7),
    'bagging_freq': hp.quniform('bagging_freq', 0.5, 2, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 70),  
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 0.2),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}

Trials_lightGBM_Completo = Trials()

In [45]:
n_iter = 1000

def F_OPT_Completa(params):
    params['max_bin'] = int(params['max_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])

    model = LGBMClassifier(
        n_estimators=n_iter,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=31,
        learning_rate=0.02,
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=-1,
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42
        )
    
    cv_Strat = StratifiedKFold(n_splits=5)
    auc_lightGBM_Completo = cross_val_score(model, X_train_subset, y_train_subset, cv=cv_Strat, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Completo, 'status': STATUS_OK}
    

In [46]:
best_lightGBM_Completo = fmin(fn=F_OPT_Completa,
                            space=space_lightGBM_completo,
                            algo=tpe.suggest,
                            max_evals=50,
                            trials=Trials_lightGBM_Completo,
                            rstate= np.random.default_rng(42)
)
best_lightGBM_Completo['max_bin'] = int(best_lightGBM_Completo['max_bin'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])
best_lightGBM_Completo['bagging_freq'] = int(best_lightGBM_Completo['bagging_freq'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])


[LightGBM] [Warning] feature_fraction is set=0.3788896147782749, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3788896147782749
[LightGBM] [Warning] min_gain_to_split is set=0.08955339629253539, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.08955339629253539
[LightGBM] [Warning] bagging_fraction is set=0.4936739623353964, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4936739623353964
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.3788896147782749, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3788896147782749
[LightGBM] [Warning] min_gain_to_split is set=0.08955339629253539, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.08955339629253539
[LightGBM] [Warning] bagging_fraction is set=0.4936739623353964, subsample=1.0 will be ignored. Current value: bagging_fracti

In [47]:
import pickle
with open("Trials_LightGBM_Completo_v2_3.pkl", "wb") as f:
    pickle.dump(Trials_lightGBM_Completo, f)
with open("Trials_lightGBM_Completo_v2_3.pkl", "rb") as f:
    Trials_lightGBM_Completo = pickle.load(f)

In [48]:
#Mostramos los resultados de los trials 
from tabulate import tabulate

def trials_to_dataframe(trials):
    """
    Convierte la info de 'trials' de Hyperopt en un DataFrame 
    con columnas relevantes (loss, hiperparámetros, etc.).
    """
    rows = []
    for trial in trials.trials:
        # trial['result']['loss'] -> la métrica
        # trial['misc']['vals'] -> diccionario de hiperparámetros propuestos
        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        # Convertir vals a algo "plano"
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    # df con columnas = keys (vals + 'loss')
    df = pd.DataFrame(rows)
    
    # A veces los hps que eran hp.quniform se quedan en listas => df['num_leaves'].apply(lambda x: x[0] if isinstance(x, list) else x)
    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)
    
    return df

# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Completo)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

+----+--------------------+----------------+--------------------+-----------+---------------------+--------------------+---------------------+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   max_bin |   min_child_samples |   min_child_weight |   min_gain_to_split |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------+---------------------+--------------------+---------------------+-------------+--------------+-----------|
| 41 |           0.67623  |              1 |           0.278413 |       280 |                  72 |         0.00912512 |           0.0504188 |   0.0385122 |     12.9891  | -0.774835 |
| 37 |           0.651814 |              2 |           0.361893 |       275 |                  69 |         0.00811064 |           0.0510952 |   0.0508589 |     14.0903  | -0.774789 |
| 18 |           0.693991 |              2 |           0.371007 |       285 |   

In [49]:
best_params = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo
    'num_leaves': 31,        # Fijo
    'learning_rate': 0.02,   # Fijo
    'max_depth': -1,         # Fijo
    'bagging_fraction': 0.67623,
    'bagging_freq': 1,
    'feature_fraction': 0.278413,
    'max_bin': 280,
    'min_child_samples': 72,
    'min_child_weight': 0.00912512,
    'min_gain_to_split':  0.0504188,
    'reg_alpha': 0.0385122,
    'reg_lambda': 12.9891,
    'scale_pos_weight': 1,   # Fijo
    'is_unbalance': False,   # Fijo
}

final_model_lgb = LGBMClassifier(
    n_estimators=1000,
    boosting_type=best_params['boosting_type'],
    objective=best_params['objective'],
    metric=best_params['metric'],
    num_leaves=best_params['num_leaves'],
    learning_rate=best_params['learning_rate'],
    max_bin=best_params['max_bin'],
    max_depth=best_params['max_depth'],
    bagging_fraction=best_params['bagging_fraction'],
    bagging_freq=best_params['bagging_freq'],   
    feature_fraction=best_params['feature_fraction'],
    min_child_samples=best_params['min_child_samples'],
    min_child_weight=best_params['min_child_weight'],
    min_gain_to_split=best_params['min_gain_to_split'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    scale_pos_weight=1,
    is_unbalance=False,
    random_state=42
)

final_model_lgb.fit(X_train_subset, y_train_subset)

[LightGBM] [Warning] feature_fraction is set=0.278413, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.278413
[LightGBM] [Warning] min_gain_to_split is set=0.0504188, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0504188
[LightGBM] [Warning] bagging_fraction is set=0.67623, subsample=1.0 will be ignored. Current value: bagging_fraction=0.67623
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.278413, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.278413
[LightGBM] [Warning] min_gain_to_split is set=0.0504188, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0504188
[LightGBM] [Warning] bagging_fraction is set=0.67623, subsample=1.0 will be ignored. Current value: bagging_fraction=0.67623
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_f

LGBMClassifier(bagging_fraction=0.67623, bagging_freq=1,
               feature_fraction=0.278413, is_unbalance=False,
               learning_rate=0.02, max_bin=280, metric='auc',
               min_child_samples=72, min_child_weight=0.00912512,
               min_gain_to_split=0.0504188, n_estimators=1000,
               objective='binary', random_state=42, reg_alpha=0.0385122,
               reg_lambda=12.9891, scale_pos_weight=1)

In [50]:
y_pred_proba = final_model_lgb.predict_proba(X_test_subset)[:, 1]
from sklearn.metrics import roc_auc_score
auc_test = roc_auc_score(y_test_subset, y_pred_proba)
print(f'AUC en test: {auc_test}')

[LightGBM] [Warning] feature_fraction is set=0.278413, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.278413
[LightGBM] [Warning] min_gain_to_split is set=0.0504188, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0504188
[LightGBM] [Warning] bagging_fraction is set=0.67623, subsample=1.0 will be ignored. Current value: bagging_fraction=0.67623
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
AUC en test: 0.777795871108371


-AUC solo con app_train:
0.7687757918575168

-AUC añadiendo bureau.csv con threshhold de 0.00015:
0.7723138876162285

-AUC con threshold de 0.0001 y 40 features:
0.7741547798044517

-AUC con threshold de 0 y ~80 features:
0.777795871108371